# REINFORCE

## Psuedocode
1. policy $\pi_\theta$를 이용해서 $m$개의 trajectory $\{ \tau^{(1)}, \tau^{(2)}, ... , \tau^{(m)} \}$를 생성한다.
   - 각 trajectory는 horizon $H$ 만큼의 길이를 갖는다.
   - $i$번째 trajectory $\tau^{(i)} = (s_0^{(i)}, a_0^{(i)}, ..., s_H^{(i)}, a_H^{(i)}, s_{H+1}^{(i)})$  
  
2. 모든 trajectory를 이용해서 gradient $\triangledown_\theta J(\theta)$를 추정한다.
   - $\triangledown_{\theta}J(\theta) \approx \hat{g} := \frac{1}{m}\Sigma_{i=1}^m\Sigma_{t=0}^H \triangledown_{\theta}\log\pi_{\theta}(a_t^{(i)} | s_t^{(i)})R(\tau^{(i)})$  
  
3. policy의 weights를 업데이트한다.
   - $\theta \leftarrow \theta + \alpha\hat{g}$
  
4. 1-3을 반복한다.

In [1]:
import gym
import numpy as np
from itertools import count

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

from model import Policy

In [2]:
env = gym.make('CartPole-v1')
env.seed(1)
torch.manual_seed(1)

In [3]:
class Policy(nn.Module):
    def __init__(self):
        super(Policy, self).__init__()
        self.affine1 = nn.Linear(4, 128)
        self.dropout = nn.Dropout(p=0.6)
        self.affine2 = nn.Linear(128, 2)

        self.saved_log_probs = []
        self.rewards = []

    def forward(self, x):
        x = self.affine1(x)
        x = self.dropout(x)
        x = F.relu(x)
        action_scores = self.affine2(x)
        return F.softmax(action_scores, dim=1)


In [38]:
def select_action(policy, state, is_train):
    # numpy로 된 state를 torch tensor화 한다.
    state = torch.from_numpy(state).float().unsqueeze(0)
    
    # state를 policy에 넣어 action의 확률을 구한다.
    if is_train:
        probs = policy(state)
    else:
        with torch.no_grad():
            probs = policy(state)
    
    # action의 확률값을 이용해 샘플링을 수행하여 액션을 뽑는다.
    # torch.distributions의 Categorical을 이용해 샘플링에 대한 최적화 수행 가능
    m = Categorical(probs)
    action = m.sample()
    
    # 해당 action의 log_probs를 저장한다.
    policy.saved_log_probs.append(m.log_prob(action))
    
    # 액션의 값을 리턴한다.
    return action.item()

In [30]:
def finish_episode(policy, optimizer, gamma, eps):
    R = 0
    policy_loss = []
    returns = []
    
    # policy가 모은 rewards의 값에 할인율 gamma를 적용한다.
    for r in policy.rewards[::-1]:
        R = r + gamma * R
        returns.insert(0, R)
    returns = torch.tensor(returns)
    
    # 리턴을 정규화한다.
    returns = (returns - returns.mean()) / (returns.std() + eps)
    
    # -log_prob * reward
    # (-)는 Gradient Descent가 아닌 Gradient Ascent이므로
    for log_prob, R in zip(policy.saved_log_probs, returns):
        policy_loss.append(-log_prob * R)
    # 개별 loss의 총합이 policy_loss가 됨
    policy_loss = torch.cat(policy_loss).sum()
        
    optimizer.zero_grad()
    policy_loss.backward()
    optimizer.step()
    del policy.rewards[:]
    del policy.saved_log_probs[:]
    
    return policy, optimizer

In [39]:
def train(policy, optimizer, gamma, eps, render):
    running_reward = 10
    
    # n번 반복한다 - 여기서는 맨 끝에 CartPole 문제가 풀릴 때까지.
    for i_episode in count(1):
        
        # 환경과 reward를 리셋
        state, ep_reward = env.reset(), 0
        
        # 게임을 플레이한다. 학습시에는 스텝은 10,000번으로 제한을 둔다.
        for t in range(1, 10000):
            
            # 현재 state에 대한 action을 선택한다.
            action = select_action(policy, state, is_train=True)
            
            # 선택한 action을 실행해서 다음 state, reward, done, info를 얻는다.
            state, reward, done, _ = env.step(action)
            
            # render option이 true인 경우 시각적으로 보여준다.
            if render:
                env.render()
                
            # reward를 policy의 rewards에 추가한다.
            policy.rewards.append(reward)
            
            # ep_reward에 더해준다.
            ep_reward += reward
            
            # 게임 오버되면 loop를 종료한다.
            if done:
                break

        running_reward = 0.05 * ep_reward + (1 - 0.05) * running_reward
        
        # 에피소드가 끝나면 policy를 업데이트한다.
        policy, optimizer = finish_episode(policy, optimizer, gamma, eps)
        
        # 로깅
        log_interval = 10
        if i_episode % log_interval == 0:
            print('Episode {}\tLast reward: {:.2f}\tAverage reward: {:.2f}'.format(
                  i_episode, ep_reward, running_reward))
        if running_reward > env.spec.reward_threshold:
            print("Solved! Running reward is now {} and "
                  "the last episode runs to {} time steps!".format(running_reward, t))
            
            torch.save(policy.state_dict(), "solved.pth")
            print("model saved.")
            break


In [40]:
RENDER = False
LR = 0.01
GAMMA = 0.99

policy = Policy()
optimizer = optim.Adam(policy.parameters(), lr=LR)
EPS = np.finfo(np.float32).eps.item() #small eps value

train(policy, optimizer, GAMMA, EPS, RENDER)

Episode 10	Last reward: 36.00	Average reward: 13.69
Episode 20	Last reward: 33.00	Average reward: 19.32
Episode 30	Last reward: 37.00	Average reward: 25.70
Episode 40	Last reward: 60.00	Average reward: 40.37
Episode 50	Last reward: 92.00	Average reward: 49.16
Episode 60	Last reward: 146.00	Average reward: 61.31
Episode 70	Last reward: 104.00	Average reward: 87.16
Episode 80	Last reward: 132.00	Average reward: 107.66
Episode 90	Last reward: 151.00	Average reward: 159.99
Episode 100	Last reward: 191.00	Average reward: 146.75
Episode 110	Last reward: 202.00	Average reward: 217.45
Episode 120	Last reward: 103.00	Average reward: 177.93
Episode 130	Last reward: 128.00	Average reward: 138.37
Episode 140	Last reward: 312.00	Average reward: 156.50
Episode 150	Last reward: 386.00	Average reward: 247.40
Episode 160	Last reward: 500.00	Average reward: 333.87
Episode 170	Last reward: 468.00	Average reward: 398.93
Episode 180	Last reward: 187.00	Average reward: 318.90
Episode 190	Last reward: 126.00

In [41]:
trained_policy = Policy()
path = "solved.pth"
trained_policy.load_state_dict(torch.load(path, map_location="cpu"))
trained_policy.eval()

Policy(
  (affine1): Linear(in_features=4, out_features=128, bias=True)
  (dropout): Dropout(p=0.6, inplace=False)
  (affine2): Linear(in_features=128, out_features=2, bias=True)
)

In [42]:
random_policy = Policy()
random_policy.eval()

Policy(
  (affine1): Linear(in_features=4, out_features=128, bias=True)
  (dropout): Dropout(p=0.6, inplace=False)
  (affine2): Linear(in_features=128, out_features=2, bias=True)
)

In [44]:
def play(env, policy):
    
    state, ep_reward = env.reset(), 0
        
    # 게임을 플레이한다.
    for t in count(1):

        # 현재 state에 대한 action을 선택한다.
        action = select_action(policy, state, is_train=False)

        # 선택한 action을 실행해서 다음 state, reward, done, info를 얻는다.
        state, reward, done, _ = env.step(action)
        
        ep_reward += reward

        # 플레이를 시각적으로 보여준다.
        env.render()
            
        if done:
            print("episode reward sum: {}".format(ep_reward))
            break
            
    env.close()
                
                
env = gym.make('CartPole-v1')
env.seed(1)
play(env, trained_policy)

episode reward sum: 500.0


In [45]:
env = gym.make('CartPole-v1')
env.seed(1)
play(env, random_policy)

episode reward sum: 21.0
